In [ ]:
# https://arize.com/blog-course/react-agent-llm/

In [ ]:
# !pip install -U duckduckgo-search

In [32]:
import warnings

from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

from langchain.agents import load_tools
from langchain.tools import ShellTool
from langchain.tools import DuckDuckGoSearchRun

from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.chains import create_history_aware_retriever

from langchain.tools.retriever import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

from langchain import hub
from langchain.prompts import PromptTemplate

from langchain.chains.combine_documents import create_stuff_documents_chain

In [2]:
# Suppress UserWarnings
warnings.filterwarnings('ignore', category=UserWarning)

In [3]:
import os
openai_api_key = "..." # https://platform.openai.com/
os.environ["OPENAI_API_KEY"] = openai_api_key

In [4]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)
tools = load_tools(['terminal'], llm=llm)

In [5]:
# https://python.langchain.com/docs/integrations/tools/
tools.append(DuckDuckGoSearchRun())

In [8]:
context_path = './context'
loader = DirectoryLoader(context_path, glob="*", loader_cls=TextLoader)
docs = loader.load()
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Print documents
print(f"Documents: {len(documents)}")
for doc_element in documents:
    print(doc_element)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)

Documents: 1
page_content='In circuits bright, where logic flows,\nA robot mused, with wires aglow.\n"Two plus three, in silicon heaven,\nIs not five, but rather seven."' metadata={'source': 'context/poem.txt'}


In [24]:
# prompt = hub.pull("rlm/rag-prompt-default")
prompt = PromptTemplate(
    input_variables=["input", "question"],
    template="""Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer. 
    Use three sentences maximum and keep the answer as concise as possible. 
    {input}
    Question: {question}
    Helpful Answer:"""
)

In [39]:
# prompt = hub.pull("smithing-gold/assumption-checker")

In [41]:
# Extracting prompt
agent_type = 'chat-conversational-react-description'
agent = initialize_agent(tools, llm, agent=agent_type, verbose=True, handle_parsing_errors=True)
prompt = agent.agent.llm_chain.prompt
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and 

In [42]:
retriever = vector.as_retriever()
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [43]:
# History
chat_history = []
# messages.append(SystemMessage(content=system_text))

In [44]:
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['agent_scratchpad', 'chat_history', 'input']

In [ ]:
response = str(retrieval_chain.invoke({
    "chat_history": messages,
    "input": user_input
})['answer'])

In [15]:
retriever_tool = create_retriever_tool(
    retriever, 
    "search_in_text_documents", 
    "Searches and returns documents regarding in the folder."
)

NameError: name 'retriever' is not defined

In [ ]:
# https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_types.AgentType.html
# agent_type = 'chat-zero-shot-react-description'
agent_type = 'chat-conversational-react-description'
agent = initialize_agent(tools, llm, agent=agent_type, verbose=True, handle_parsing_errors=True)

In [ ]:
# agent.agent.llm_chain.prompt.messages[0].prompt.template

In [ ]:
agent.run(input="Do you know the capital of Britain?", chat_history=chat_history)

In [ ]:
agent.run(
    input="Check the date and the list of the current directory. Do we have a file that contins a name of Capital of Britain?", 
    chat_history=chat_history
)